In [ ]:
# Distlearn Jupyter example

In [3]:
from __future__ import print_function

In [4]:
import client

In [5]:
#reload(client)

In [6]:
from sklearn.datasets import load_iris
from sklearn.cross_validation import train_test_split
iris = load_iris()
X = iris.data
Y = iris.target
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [7]:
c = client.Client("mike")
c.send_data('iris_train', {'data': X_train, 'target': y_train})
c.send_data('iris_test', {'data': X_test, 'target': y_test})

[2016-05-11 01:53:17,943] [INFO] This is client mike


INFO:client:This is client mike


[2016-05-11 01:53:17,947] [INFO] Connected to master


INFO:client:Connected to master


In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from dl_utils import Task
lr = LogisticRegression(C=1., penalty='l1')
task = Task(data='iris_train', description='LR for iris', test_data='iris_test', estimator=lr, 
            type='fit_predict', result='score', scoring=mean_squared_error)
task_id = c.send_task(task)
print(task_id, task.id)

0 0


In [9]:
result = c.collect_task(task.id)
result

0.02

In [10]:
result = c.send_task(task, async=False)
result

0.02

In [11]:
from sklearn.cross_validation import StratifiedKFold
cv = StratifiedKFold(y_train, 5)
result = c.cross_validate(data='iris_train', description='cv iris', estimator=lr, scoring=mean_squared_error, cv=cv, async=False)
result

[0.047619047619047616,
 0.10000000000000001,
 0.10000000000000001,
 0.050000000000000003,
 0.0]

In [12]:
map(print, c.list_tasks());

0 "LR for iris" user 1, failed 0, completed 1, canceled 0
1 "LR for iris" user 1, failed 0, completed 1, canceled 0
2 "cv iris" user 1, failed 0, completed 1, canceled 0


In [13]:
from sklearn.tree import DecisionTreeClassifier
est1 = DecisionTreeClassifier(max_depth=10)
est2 = DecisionTreeClassifier(criterion='entropy', max_depth=2)
result = c.stacking(data='iris_train', description='stacking test', 
                    estimators=[est1, est2], test_data='iris_test', 
                    estimator=lr, cv=cv, async=False)

In [14]:
mean_squared_error(y_test, result)

0.02

In [15]:
fail_task_id = c.stacking(data='iris_train', description='failing', 
                    estimators=[est1, est2], test_data=1, 
                    estimator=lr, cv=cv)

In [18]:
map(print, c.check_errors());

(0, (33, 'local_worker', "Error: 'int' object has no attribute '__getitem__'"))
(1, (34, 'local_worker', "Error: 'int' object has no attribute '__getitem__'"))


In [19]:
map(print, c.list_tasks(failed=True, auto=None));

22 "failing" user 1, failed 1, completed 0, canceled 0
33 "failing, phase 2,  estimator 0" user 0, failed 1, completed 0, canceled 0
34 "failing, phase 2,  estimator 1" user 0, failed 1, completed 0, canceled 0


In [21]:
c.get_task(fail_task_id)

<Task id=22, owner=mike>

In [22]:
c.cancel_task(fail_task_id)

In [23]:
map(print, c.list_tasks(auto=None));

0 "LR for iris" user 1, failed 0, completed 1, canceled 0
1 "LR for iris" user 1, failed 0, completed 1, canceled 0
2 "cv iris" user 1, failed 0, completed 1, canceled 0
3 "cv iris, fold 0" user 0, failed 0, completed 1, canceled 0
4 "cv iris, fold 1" user 0, failed 0, completed 1, canceled 0
5 "cv iris, fold 2" user 0, failed 0, completed 1, canceled 0
6 "cv iris, fold 3" user 0, failed 0, completed 1, canceled 0
7 "cv iris, fold 4" user 0, failed 0, completed 1, canceled 0
8 "stacking test" user 1, failed 0, completed 1, canceled 0
9 "stacking test, phase 1, fold 0, estimator 0" user 0, failed 0, completed 1, canceled 0
10 "stacking test, phase 1, fold 0, estimator 1" user 0, failed 0, completed 1, canceled 0
11 "stacking test, phase 1, fold 1, estimator 0" user 0, failed 0, completed 1, canceled 0
12 "stacking test, phase 1, fold 1, estimator 1" user 0, failed 0, completed 1, canceled 0
13 "stacking test, phase 1, fold 2, estimator 0" user 0, failed 0, completed 1, canceled 0
14 "st

In [24]:
mean_squared_error(y_test, c.collect_task(result))

0.02

In [91]:
b = True
a = {'local_worker': 1462919959.84}
(len(a) > 0) == b

True

In [121]:
from Queue import Queue

q = Queue()
q.queue.append(1)
q.queue.appendleft(2)
map(q.queue.appendleft, reversed([4, 3]))
print(q.queue)

deque([4, 3, 2, 1])
